In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
# optuna経由でLightGBMをインポート
import optuna.integration.lightgbm as lgb_o

In [2]:
# csvファイルからPandas DataFrameへ読み込み
train = pd.read_csv('train.csv', delimiter=',', low_memory=False)

# 冒頭を表示して確認
train.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,target
0,0,0,0,6,1,0,0,0,0,7,...,0,0,0,0,0,0,2,0,0,Class_6
1,1,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,1,0,Class_6
2,2,0,0,0,0,0,1,0,3,0,...,0,0,0,0,1,0,0,0,0,Class_2
3,3,0,0,7,0,1,5,2,2,0,...,0,4,0,2,2,0,4,3,0,Class_8
4,4,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Class_2


In [3]:
# trainのtargetをカテゴリーに変換
train.target = train.target.astype('category')

train.dtypes

id               int64
feature_0        int64
feature_1        int64
feature_2        int64
feature_3        int64
                ...   
feature_71       int64
feature_72       int64
feature_73       int64
feature_74       int64
target        category
Length: 77, dtype: object

In [4]:
# ラベルエンコーディング（LabelEncoder）
le = LabelEncoder()
encoded = le.fit_transform(train.target.values)
decoded = le.inverse_transform(encoded)
train.target = encoded

# 冒頭を表示して確認
train.target.head()

0    5
1    5
2    1
3    7
4    1
Name: target, dtype: int64

In [5]:
# 訓練データとテストデータに分割
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(train, test_size=0.2, random_state = 0)

# 表示して確認
(train_set.shape, test_set.shape)

((160000, 77), (40000, 77))

In [6]:
# 訓練データとラベルを分割する
X_train, y_train = train_set.drop(['target'], axis=1).drop(['id'], axis=1).values, train_set.target.values

# 評価データとラベルを分割する
X_test, y_test = test_set.drop(['target'], axis=1).drop(['id'], axis=1).values, test_set.target.values

# 表示して確認
(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

((160000, 75), (160000,), (40000, 75), (40000,))

In [7]:
# データをセット
# 訓練データ
lgb_train = lgb_o.Dataset(X_train, y_train)
# 評価データ
lgb_eval = lgb_o.Dataset(X_test, y_test)

In [8]:
# lightgbmを実装して識別を行う--------------------------------------------------

# ハイパーパラメータサーチ&モデル構築
params = {'objective': 'multiclass',
          'num_class': 9,
          'metric': 'multi_error',
          'random_seed': 0} 

# Optunaでのパラメータ探索
gbm_o = lgb_o.train(params,
                    lgb_train,
                    valid_sets=lgb_eval,
                    early_stopping_rounds=100,
                    verbose_eval=100,)

[I 2021-06-30 00:09:48,038] A new study created in memory with name: no-name-f35b93c2-5954-454f-9c2b-e96ec8402c09
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.063629 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3288
[LightGBM] [Info] Number of data points in the train set: 160000, number of used features: 75
[LightGBM] [Info] Start training from score -3.086615
[LightGBM] [Info] Start training from score -2.096791
[LightGBM] [Info] Start training from score -2.601750
[LightGBM] [Info] Start training from score -3.752885
[LightGBM] [Info] Start training from score -4.183176
[LightGBM] [Info] Start training from score -1.351507
[LightGBM] [Info] Start training from score -2.606566
[LightGBM] [Info] Start training from score -1.353174
[LightGBM] [Info] Start training from score -2.058904
Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_error: 0.638325


feature_fraction, val_score: 0.636475:  14%|#4        | 1/7 [00:12<01:17, 12.84s/it]

Early stopping, best iteration is:
[55]	valid_0's multi_error: 0.636475
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.065698 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3288
[LightGBM] [Info] Number of data points in the train set: 160000, number of used features: 75
[LightGBM] [Info] Start training from score -3.086615
[LightGBM] [Info] Start training from score -2.096791
[LightGBM] [Info] Start training from score -2.601750
[LightGBM] [Info] Start training from score -3.752885
[LightGBM] [Info] Start training from score -4.183176
[LightGBM] [Info] Start training from score -1.351507
[LightGBM] [Info] Start training from score -2.606566
[LightGBM] [Info] Start training from score -1.353174
[LightGBM] [Info] Start training from score -2.058904
Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_e

feature_fraction, val_score: 0.635950:  29%|##8       | 2/7 [00:27<01:06, 13.32s/it]

...

Early stopping, best iteration is:
[109]	valid_0's multi_error: 0.6356
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.072012 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3288
[LightGBM] [Info] Number of data points in the train set: 160000, number of used features: 75
[LightGBM] [Info] Start training from score -3.086615
[LightGBM] [Info] Start training from score -2.096791
[LightGBM] [Info] Start training from score -2.601750
[LightGBM] [Info] Start training from score -3.752885
[LightGBM] [Info] Start training from score -4.183176
[LightGBM] [Info] Start training from score -1.351507
[LightGBM] [Info] Start training from score -2.606566
[LightGBM] [Info] Start training from score -1.353174
[LightGBM] [Info] Start training from score -2.058904
Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_er

min_data_in_leaf, val_score: 0.633975:  80%|########  | 4/5 [00:51<00:13, 13.03s/it]

Early stopping, best iteration is:
[62]	valid_0's multi_error: 0.634675
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.070621 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3288
[LightGBM] [Info] Number of data points in the train set: 160000, number of used features: 75
[LightGBM] [Info] Start training from score -3.086615
[LightGBM] [Info] Start training from score -2.096791
[LightGBM] [Info] Start training from score -2.601750
[LightGBM] [Info] Start training from score -3.752885
[LightGBM] [Info] Start training from score -4.183176
[LightGBM] [Info] Start training from score -1.351507
[LightGBM] [Info] Start training from score -2.606566
[LightGBM] [Info] Start training from score -1.353174
[LightGBM] [Info] Start training from score -2.058904
Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_e

min_data_in_leaf, val_score: 0.633975: 100%|##########| 5/5 [01:03<00:00, 12.60s/it]

Early stopping, best iteration is:
[63]	valid_0's multi_error: 0.6352


In [9]:
# ベストパラメータの取得
best_params = gbm_o.params
print("  Params: ")
for key, value in best_params.items():
    print("    {}: {}".format(key, value))

  Params: 
    objective: multiclass
    num_class: 9
    metric: multi_error
    random_seed: 0
    feature_pre_filter: False
    lambda_l1: 0.0
    lambda_l2: 0.0
    num_leaves: 19
    feature_fraction: 0.7
    bagging_fraction: 1.0
    bagging_freq: 0
    min_child_samples: 20
    num_iterations: 1000
    early_stopping_round: 100


In [10]:
# 調整後モデルで予測の実行
Y_pred = gbm_o.predict(X_test, num_iteration = gbm_o.best_iteration)

# 予測確率を整数へ
y_pred = np.argmax(Y_pred, axis=1)

# 識別率を求める
score = accuracy_score(y_test, y_pred)
print(f'score: {score}')

score: 0.366025


In [11]:
from sklearn.metrics import confusion_matrix
# 混同行列を表示
cmx = confusion_matrix(y_test, y_pred)
cmx

array([[   0,  436,    1,    1,    0,  457,    0,  909,    9],
       [   0, 2376,    6,    0,    1,  992,    2, 1368,   30],
       [   0, 1127,    3,    1,    2,  698,    0, 1084,   20],
       [   0,  226,    2,    0,    0,  257,    0,  459,    8],
       [   0,  129,    0,    1,    0,  156,    0,  330,    8],
       [   1,  857,    3,    0,    0, 5408,    0, 4105,   21],
       [   1,  317,    1,    1,    0,  789,    0, 1842,   12],
       [   1,  781,    8,    1,    0, 2750,    2, 6830,   43],
       [   0, 1177,    6,    1,    1, 1384,    2, 2532,   24]])

In [12]:
# csvファイルからPandas DataFrameへ読み込み
submit_data = pd.read_csv('test.csv', delimiter=',', low_memory=False)

# 冒頭を表示して確認
submit_data.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74
0,200000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,200001,1,2,0,0,0,0,0,0,0,...,3,1,3,0,0,0,0,3,0,0
2,200002,0,1,7,1,0,0,0,0,6,...,3,0,0,0,0,3,0,2,0,0
3,200003,0,0,0,4,3,1,0,0,0,...,0,0,0,1,0,0,0,4,0,0
4,200004,0,0,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [13]:
# 提出データを適用できる形にする
x_submit = submit_data.drop(['id'], axis=1)

# 冒頭を表示して確認
x_submit.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,0,0,0,0,0,0,0,0,...,3,1,3,0,0,0,0,3,0,0
2,0,1,7,1,0,0,0,0,6,0,...,3,0,0,0,0,3,0,2,0,0
3,0,0,0,4,3,1,0,0,0,0,...,0,0,0,1,0,0,0,4,0,0
4,0,0,5,0,0,0,0,0,0,8,...,0,0,0,0,0,0,0,0,1,0


In [14]:
# 提出データを予測する
predictions = gbm_o.predict(x_submit, num_iteration = gbm_o.best_iteration)

# データの形を確認
predictions.shape

(100000, 9)

In [15]:
# 結果を提出形式に変形
df_predictions = pd.DataFrame(predictions)
df_predictions.columns = ['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8', 'Class_9']
submit_data = pd.concat([submit_data.id,df_predictions],axis=1)
submit_data = submit_data.set_index('id')
submit_data.head()

,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
id,,,,,,,,,
200000,0.047166,0.386004,0.163520,0.025399,0.013095,0.173671,0.025002,0.053809,0.112334
200001,0.044906,0.084482,0.087967,0.020954,0.011719,0.221131,0.080786,0.322586,0.125470
200002,0.028194,0.033264,0.024879,0.013417,0.008328,0.672826,0.034104,0.128190,0.056798
200003,0.057256,0.082436,0.082390,0.048024,0.015027,0.248217,0.062445,0.241977,0.162228
200004,0.037939,0.116377,0.082335,0.028219,0.014011,0.285166,0.054686,0.235892,0.145374


In [16]:
# CSVファイルとして出力
submit_data.to_csv("submission_LGBM_optuna.csv")